In [ ]:
!pip install transformers datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24

FINETUNING


In [35]:
pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 5.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [3]:
pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 5.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [1]:
!pip show accelerate

Name: accelerate
Version: 0.31.0
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: zach.mueller@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: 


In [4]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, ProgressCallback
import torch
from sklearn.metrics import accuracy_score, classification_report

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

# Load your dataset
df_test = pd.read_csv('data/original_dataset/test_hate_dataset.csv')
df_train = pd.read_csv('train_hate_dataset.csv')

# Create binary labels: 0 for non-hate speech (offensive and neither), 1 for hate speech
df_test['label'] = df_test['label'].apply(lambda x: 1 if x == 'hate' else 0)
df_train['label'] = df_train['label'].apply(lambda x: 1 if x == 'hate' else 0)

train_texts = df_train['text']
train_labels = df_train['label']
test_texts = df_test['text']
test_labels = df_test['label']

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('cardiffnlp/twitter-roberta-base-hate')
model = AutoModelForSequenceClassification.from_pretrained('cardiffnlp/twitter-roberta-base-hate').to(device)

# Tokenize the texts
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=512)

# Create a Dataset class
class HateSpeechDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}  # Convert lists to tensors
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create the datasets
train_dataset = HateSpeechDataset(train_encodings, train_labels.values) # Remove .to(device) here
test_dataset = HateSpeechDataset(test_encodings, test_labels.values) # Remove .to(device) here

# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # number of training epochs
    per_device_train_batch_size=16,  # batch size for training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    evaluation_strategy="epoch"      # evaluate at the end of each epoch
)

# Define the Trainer
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,           # evaluation dataset
    callbacks=[ProgressCallback()]       # list of callbacks to apply during training
)

# Train the model
trainer.train()



Using device: cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/4629 [00:00<?, ?it/s]

Epoch,Training Loss,Validation Loss
1,0.399600,0.411626


{'loss': 0.9495, 'grad_norm': 17.97427749633789, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.01}
{'loss': 0.8864, 'grad_norm': 6.453651428222656, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.01}
{'loss': 0.7309, 'grad_norm': 13.393362045288086, 'learning_rate': 3e-06, 'epoch': 0.02}
{'loss': 0.5941, 'grad_norm': 5.61953592300415, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.03}
{'loss': 0.713, 'grad_norm': 7.293231010437012, 'learning_rate': 5e-06, 'epoch': 0.03}
{'loss': 0.6369, 'grad_norm': 7.0598673820495605, 'learning_rate': 6e-06, 'epoch': 0.04}
{'loss': 0.6323, 'grad_norm': 6.552701950073242, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.05}
{'loss': 0.6226, 'grad_norm': 7.004067420959473, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.05}
{'loss': 0.5819, 'grad_norm': 6.716711044311523, 'learning_rate': 9e-06, 'epoch': 0.06}
{'loss': 0.5833, 'grad_norm': 8.41675853729248, 'learning_rate': 1e-05, 'epoch': 0.06}
{'loss': 0.5389, 'grad_norm': 13.221358

  0%|          | 0/1029 [00:00<?, ?it/s]

{'eval_loss': 0.4116261303424835, 'eval_runtime': 532.1903, 'eval_samples_per_second': 30.925, 'eval_steps_per_second': 1.934, 'epoch': 1.0}
{'loss': 0.4123, 'grad_norm': 3.898081064224243, 'learning_rate': 3.7285056914507144e-05, 'epoch': 1.0}
{'loss': 0.3529, 'grad_norm': 24.539804458618164, 'learning_rate': 3.716396221845483e-05, 'epoch': 1.01}
{'loss': 0.335, 'grad_norm': 7.015546798706055, 'learning_rate': 3.704286752240252e-05, 'epoch': 1.02}
{'loss': 0.2675, 'grad_norm': 10.729352951049805, 'learning_rate': 3.692177282635021e-05, 'epoch': 1.02}
{'loss': 0.3002, 'grad_norm': 20.517486572265625, 'learning_rate': 3.6800678130297894e-05, 'epoch': 1.03}
{'loss': 0.3532, 'grad_norm': 12.07894229888916, 'learning_rate': 3.6679583434245576e-05, 'epoch': 1.04}
{'loss': 0.2994, 'grad_norm': 11.918972969055176, 'learning_rate': 3.655848873819327e-05, 'epoch': 1.04}
{'loss': 0.3656, 'grad_norm': 6.500513076782227, 'learning_rate': 3.6437394042140954e-05, 'epoch': 1.05}
{'loss': 0.4027, 'gra

KeyboardInterrupt: 

In [5]:
# Evaluate the model
predictions, labels, _ = trainer.predict(test_dataset)
predictions = torch.argmax(torch.tensor(predictions), dim=1)

# Calculate accuracy
accuracy = accuracy_score(test_labels, predictions.cpu().numpy())
print(f'Accuracy: {accuracy:.4f}')

# Print detailed classification report
print(classification_report(test_labels, predictions.cpu().numpy(), target_names=['Non-Hate Speech', 'Hate Speech']))


  0%|          | 0/1029 [00:00<?, ?it/s]

Epoch,Training Loss,Validation Loss
1,0.399600,0.411626


Accuracy: 0.8239
                 precision    recall  f1-score   support

Non-Hate Speech       0.77      0.88      0.82      7625
    Hate Speech       0.88      0.77      0.83      8833

       accuracy                           0.82     16458
      macro avg       0.83      0.83      0.82     16458
   weighted avg       0.83      0.82      0.82     16458



In [6]:
df = pd.read_csv('data/original_dataset/test_hate_dataset.csv')

print( df['label'].value_counts()['nothate'])
df['prediction'] = predictions
df.head()
df.to_csv('data/classified_sentences/classified_sentences_finetuned.csv', index=False)

7625
